# Image Colourisation with Neural Networks

This example is about automatically colourising images from greyscale sources. Details here: http://www.vision.caltech.edu/Image_Datasets/Caltech101/

Citation: L. Fei-Fei, R. Fergus and P. Perona. Learning generative visual models from few training examples: an incremental Bayesian approach tested on 101 object categories. IEEE. CVPR 2004, Workshop on Generative-Model Based Vision. 2004

We will be using Keras to build a convolutional neural network and colourise greyscale images

As with all of the toy builds, this code will follow the same structure:  

    Importing Packages
    Preparing the Data  
    Building the Neural Network  
    Testing the Neural Network   

## Importing Packages

In [5]:
# Reading in image files
import os

# Preparing the Data
import numpy as np 
from PIL import ImageOps, Image # Equalise the image
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import random

# Building and Testing the Network
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import *
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

## Preparing the Data

In [2]:
root_path = "/home/de-admin/Documents/github folders/Mike Thomas/Neural-Network-Toy-Builds/Image Colourising/Caltech-256"

images  = []
for directory in os.listdir(root_path):
    for image in os.listdir(root_path+"/"+directory):
        images.append(root_path+"/"+directory+"/"+image)

def process_image(image_path):
    img_in = Image.open(image_path)
    img_in_r = img_in.resize((300,300))
    
    img_equalised = ImageOps.equalize(img_in_r, mask=None)
    img_equalised_grey = img_equalised.convert('L')
    
    img_final = np.array(img_equalised, dtype=np.float32)
    img_final_grey = np.array(img_equalised_grey, dtype=np.float32)
    
    return img_final_grey, img_final

all_colour = []
all_greyscale = []

for image in images:
    img_final_grey, img_final = process_image(image)
    if img_final_grey.shape == (300,300):
        if img_final.shape == (300,300,3):         
            all_greyscale.append(img_final_grey)
            all_colour.append(img_final)

X_train = []
X_test = []
y_train = []
y_test = []

for i in range(len(all_greyscale)):
    if random.uniform(0,1) > 0.1:
        X_train.append(all_greyscale[i])
        y_train.append(all_colour[i])
    else:
        X_test.append(all_greyscale[i])
        y_test.append(all_colour[i])

X_train = np.asarray(X_train)/255
y_train = np.asarray(y_train)/255
X_test = np.asarray(X_test)/255
y_test = np.asarray(y_test)/255

## Building the Neural Network

In [3]:
#This code is largely borrowed from Keras' MNIST example.

batch_size = 32
epochs = 2

img_rows, img_cols = 300, 300

x_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
x_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'Training samples')
print(x_test.shape[0], 'Testing samples')

x_train shape: (7890, 300, 300, 1)
y_train shape: (7890, 300, 300, 3)
7890 Training samples
844 Testing samples


In [10]:
model = Sequential()

model.add(Dense(3, input_shape=input_shape)) # Makes the output shape (300,300,3)

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Train on 7890 samples, validate on 844 samples
Epoch 1/2
7890/7890 [==============================] - 428s 54ms/step - loss: nan - acc: 0.4473 - val_loss: nan - val_acc: 0.4435
Epoch 2/2
7890/7890 [==============================] - 425s 54ms/step - loss: nan - acc: 0.4481 - val_loss: nan - val_acc: 0.4435


## Testing the Neural Network

### Test Data Set

In [11]:
print("Testing on test data:")
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Testing on test data:
Test loss: nan
Test accuracy: 0.4435101642992824
